In [1]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

In [2]:
def get_data(file):
    x = []
    y = []
    with open(file, 'r', encoding='utf8') as f:
        for line in f:
            i=14
            while(line[i]!=' '):
                i=i+1
            # lấy nhãn và nội dung
            label=line[14:i]
            content=line[i+1:]
            # xóa ký tự đặc biệt và tách từ
            content = gensim.utils.simple_preprocess(content)
            content = ' '.join(content)
            content = ViTokenizer.tokenize(content)
            x.append(content)
            y.append(label)
            
    return x,y
train_file = 'train2.txt'
x_data, y_data = get_data(train_file)
print(y_data)

['khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ba

In [3]:
import pickle
# lưu dữ liệu
pickle.dump(x_data, open('data/x_data2.pkl', 'wb'))
pickle.dump(y_data, open('data/y_data2.pkl', 'wb'))

In [13]:
test_file = 'test2.txt'
x_test, y_test = get_data(test_file)
print(y_test)
pickle.dump(x_test, open('data/x_test2.pkl', 'wb'))
pickle.dump(y_test, open('data/y_test2.pkl', 'wb'))

['khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'ban_hang', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'kh

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import pickle

x_data = pickle.load(open('data/x_data.pkl', 'rb'))
y_data = pickle.load(open('data/y_data.pkl', 'rb'))

x_test = pickle.load(open('data/x_test.pkl', 'rb'))
y_test = pickle.load(open('data/y_test.pkl', 'rb'))

# trích xuất đặc trưng

# phương pháp tf-idf ngram level - số lượng từ đại diện tối đa 5000
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=5000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(x_data)

x_data_tfidf_ngram =  tfidf_vect_ngram.transform(x_data)
x_test_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)

# giảm chiều của dữ liệu
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(x_data_tfidf_ngram)

x_data_tfidf_ngram_svd = svd_ngram.transform(x_data_tfidf_ngram)
x_test_tfidf_ngram_svd = svd_ngram.transform(x_test_tfidf_ngram)

In [2]:
from sklearn.metrics import accuracy_score, classification_report
def train_model(classifier, x_data, y_data, x_test, y_test):
    
    classifier.fit(x_data, y_data)
    
    train_pred = classifier.predict(x_data)
        #val_predictions = classifier.predict(x_val)
    test_pred = classifier.predict(x_test)
    print("Test accuracy: ", accuracy_score(test_pred, y_test))
    print(classification_report(y_test, test_pred))
    for label in test_pred:
        print(label)

In [ ]:
from sklearn.svm import SVC
train_model(SVC(gamma=2), x_data_tfidf_ngram_svd, y_data, x_test_tfidf_ngram_svd, y_test)

In [6]:
from sklearn.naive_bayes import BernoulliNB
train_model(BernoulliNB(), x_data_tfidf_ngram, y_data, x_test_tfidf_ngram, y_test)

Test accuracy:  0.940676553907682
              precision    recall  f1-score   support

    ban_hang       0.85      0.65      0.74       766
   khong_ban       0.95      0.98      0.97      5235

   micro avg       0.94      0.94      0.94      6001
   macro avg       0.90      0.82      0.85      6001
weighted avg       0.94      0.94      0.94      6001

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_ha

khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
k

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khon

In [4]:
from sklearn.linear_model import LogisticRegression
train_model(LogisticRegression(solver='lbfgs'), x_data_tfidf_ngram, y_data, x_test_tfidf_ngram, y_test)

Test accuracy:  0.955174137643726
              precision    recall  f1-score   support

    ban_hang       0.95      0.69      0.80       766
   khong_ban       0.96      0.99      0.97      5235

   micro avg       0.96      0.96      0.96      6001
   macro avg       0.95      0.84      0.89      6001
weighted avg       0.95      0.96      0.95      6001

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_

khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
kh

ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ba

In [5]:
from sklearn.naive_bayes import MultinomialNB
train_model(MultinomialNB(), x_data_tfidf_ngram, y_data, x_test_tfidf_ngram, y_test)

Test accuracy:  0.961173137810365
              precision    recall  f1-score   support

    ban_hang       0.93      0.75      0.83       766
   khong_ban       0.96      0.99      0.98      5235

   micro avg       0.96      0.96      0.96      6001
   macro avg       0.95      0.87      0.90      6001
weighted avg       0.96      0.96      0.96      6001

khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban

khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
kho

khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
ban_hang
khong_ban
ban_hang
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
khong_ban
k